In [66]:
models_d1 = ["skywalker290/Timesformers-d1","skywalker290/videomae-base-finetuned-ucf101-subset","skywalker290/Timesformer-Timesformer-d1","skywalker290/Timesformer-vivit-d1"]
models_d2 = ["skywalker290/VIVIT-d2","skywalker290/Timesformers-d2","skywalker290/Videomae-d2"]

In [67]:
from transformers import pipeline, AutoProcessor, VivitImageProcessor,VideoMAEImageProcessor
image_processors = [
                    [AutoProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400"),"timesformer"],
                    [VivitImageProcessor.from_pretrained("google/vivit-b-16x2-kinetics400"),"vivit"],
                    [VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base"),"videomae"]
                   ]

In [72]:
from transformers import pipeline, AutoProcessor, VivitImageProcessor,VideoMAEImageProcessor

# Load the processor explicitly for ViViT
image_processor = AutoProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")
#image_processor = VivitImageProcessor.from_pretrained("google/vivit-b-16x2-kinetics400")
#image_processor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base")

# Load the pipeline with the processor
for model in models_d2:
    for image_processor in image_processors:
        video_cls = pipeline(
            task="video-classification",
            model=model,
            device=0,
            image_processor=image_processor[0]
        )
        result = video_cls("d1/Anomaly-detection-Dataset/test/Anomaly/Abuse008_x264.mp4",top_k=13)
        print(image_processor[1],model.split('/')[-1],result)
        break
    break


[{'score': 0.7822444438934326, 'label': 'Abuse'}, {'score': 0.04848545789718628, 'label': 'Arson'}, {'score': 0.029100557789206505, 'label': 'Burglary'}, {'score': 0.028527015820145607, 'label': 'Shooting'}, {'score': 0.026359234005212784, 'label': 'Stealing'}, {'score': 0.016270173713564873, 'label': 'Vandalism'}, {'score': 0.015732167288661003, 'label': 'Assault'}, {'score': 0.01432537380605936, 'label': 'Explosion'}, {'score': 0.010794148780405521, 'label': 'Arrest'}, {'score': 0.008485277183353901, 'label': 'Fighting'}, {'score': 0.007636232767254114, 'label': 'Robbery'}, {'score': 0.007446467410773039, 'label': 'Shoplifting'}, {'score': 0.004593418911099434, 'label': 'RoadAccidents'}]


In [33]:
image_processor = AutoProcessor.from_pretrained("skywalker290/Videomae-d2")

preprocessor_config.json:   0%|          | 0.00/415 [00:00<?, ?B/s]

In [11]:
def run_inference(model, video):
    # (num_frames, num_channels, height, width)
    perumuted_sample_test_video = video.permute(1, 0, 2, 3)
    inputs = {
        "pixel_values": perumuted_sample_test_video.unsqueeze(0),
        "labels": torch.tensor(
            [sample_test_video["label"]]
        ),  # this can be skipped if you don't have labels available.
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model = model.to(device)

    # forward pass
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    return logits

In [13]:
from transformers import AutoImageProcessor, TimesformerForVideoClassification

model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")
rlogits = run_inference(model, "d1/Anomaly-detection-Dataset/test/Anomaly/Abuse008_x264.mp4")

AttributeError: 'str' object has no attribute 'permute'

In [3]:
vivit-videomae

SyntaxError: invalid syntax (3571890592.py, line 1)

In [25]:
import pytorchvideo.data
import torchvision

dataset_root_path = 'd1/Anomaly-detection-Dataset'
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize,
)
mean = image_processor.image_mean
std = image_processor.image_std
if "shortest_edge" in image_processor.size:
    height = width = image_processor.size["shortest_edge"]
else:
    height = image_processor.size["height"]
    width = image_processor.size["width"]
resize_to = (height, width)

num_frames_to_sample = model.config.num_frames
sample_rate = 4
fps = 30
clip_duration = num_frames_to_sample * sample_rate / fps

train_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(num_frames_to_sample),
                    Lambda(lambda x: x / 255.0),
                    Normalize(mean, std),
                    RandomShortSideScale(min_size=256, max_size=320),
                    RandomCrop(resize_to),
                    RandomHorizontalFlip(p=0.5),
                ]
            ),
        ),
    ]
)

train_dataset = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "train"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("random", clip_duration),
    decode_audio=False,
    transform=train_transform,
)
val_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(num_frames_to_sample),
                    Lambda(lambda x: x / 255.0),
                    Normalize(mean, std),
                    Resize(resize_to),
                ]
            ),
        ),
    ]
)

val_dataset = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "val"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)

test_dataset = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "test"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)

In [63]:
print(dir(val_dataset))
print(test_dataset.video_path_handler)

['_MAX_CONSECUTIVE_FAILURES', '__abstractmethods__', '__add__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_clip_sampler', '_decode_audio', '_decoder', '_is_protocol', '_labeled_videos', '_loaded_clip', '_loaded_video_label', '_next_clip_start_time', '_transform', '_video_random_generator', '_video_sampler', '_video_sampler_iter', 'num_videos', 'video_path_handler', 'video_sampler']


In [27]:
def run_inference(model, video):
    # (num_frames, num_channels, height, width)
    perumuted_sample_test_video = video.permute(1, 0, 2, 3)
    inputs = {
        "pixel_values": perumuted_sample_test_video.unsqueeze(0),
        "labels": torch.tensor(
            [sample_test_video["label"]]
        ),  # this can be skipped if you don't have labels available.
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model = model.to(device)

    # forward pass
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    return logits




In [56]:
test_dataset


In [33]:
logits

tensor([[ 3.5173, -3.8043]], device='cuda:0')

In [34]:
clip_sampler = pytorchvideo.data.make_clip_sampler("uniform", clip_duration)


In [48]:
import torch
logits = run_inference(model, sample_test_video["video"])
predicted_class_idx = logits.argmax(-1).item()
# Your tensor
logits = torch.tensor([[3.5173, -3.8043]], device='cuda:0')

# Apply softmax
probs = torch.nn.functional.softmax(logits, dim=-1)
print(probs)



tensor([[9.9934e-01, 6.6067e-04]], device='cuda:0')


In [81]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

def load_and_preprocess_data(csv_file):
    # Load the dataset
    df = pd.read_csv(csv_file)
    df = df.drop('path_to_video', axis=1)

    # Check for missing values and handle them
    df.fillna(0, inplace=True)  # Replace missing values with 0 (or another suitable method)

    # Feature columns (all except 'label' column)
    features = df.drop('label', axis=1)

    # Target column
    target = df['label']

    # Encode labels (if 'label' is categorical)
    label_encoder = LabelEncoder()
    target = label_encoder.fit_transform(target)

    # Feature scaling
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    return features_scaled, target

def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    models = {
        'Logistic Regression': LogisticRegression(),
        'Random Forest': RandomForestClassifier(),
        'SVM': SVC(),
        'KNN': KNeighborsClassifier(),
        'XGBoost': XGBClassifier()
    }

    for model_name, model in models.items():
        print(f"Training {model_name}...")
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate and print accuracy
        accuracy = accuracy_score(y_test, y_pred)
        print(f"{model_name} Accuracy: {accuracy}")

def main(csv_file):
    # Load and preprocess the data
    features_scaled, target = load_and_preprocess_data(csv_file)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

    # Train and evaluate all models
    train_and_evaluate_models(X_train, X_test, y_train, y_test)

if __name__ == "__main__":
    # Take the CSV filename as input
    csv_file = "d2_model_dataset.csv"
    main(csv_file)


Training Logistic Regression...
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Logistic Regression Accuracy: 0.5052631578947369
Training Random Forest...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Random Forest Accuracy: 0.5473684210526316
Training SVM...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
SVM Accuracy: 0.5157894736842106
Training KNN...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
KNN Accuracy: 0.47368421052631576
Training XGBoost...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END max_depth=20, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END .................................C=0.1, gamma=scale; total time=   0.0s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   2.6s
[CV] END ...learning_rate=0.3, max_depth=5, n_estimators=200; total time=   1.3s
[CV] END max_depth=10, min_samples_split=2, n_estimators=300; total time=   1.3s
[CV] END ..learning_ra

KeyboardInterrupt: 

In [87]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

# Load and preprocess the data
def load_and_preprocess_data(csv_file):
    # Load the dataset
    df = pd.read_csv(csv_file)
    df = df.drop('path_to_video', axis=1)

    # Check for missing values and handle them
    df.fillna(0, inplace=True)  # Replace missing values with 0 (or another suitable method)

    # Feature columns (all except 'label' column)
    features = df.drop('label', axis=1)

    # Target column
    target = df['label']

    # Encode labels (if 'label' is categorical)
    label_encoder = LabelEncoder()
    target = label_encoder.fit_transform(target)

    # Feature scaling
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    return features_scaled, target

# Define a simple neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input to hidden layer
        self.fc2 = nn.Linear(hidden_size, output_size)  # Hidden to output layer
        self.relu = nn.ReLU()  # Activation function
        self.softmax = nn.Softmax(dim=1)  # Softmax for multi-class classification

    def forward(self, x):
        x = self.fc1(x)  # Pass input through first layer
        x = self.relu(x)  # Apply activation function
        x = self.fc2(x)  # Pass through second layer
        x = self.softmax(x)  # Apply softmax to get probabilities
        return x

# Train and evaluate the neural network
def train_and_evaluate_neural_network(X_train, X_test, y_train, y_test, input_size, output_size):
    # Convert to PyTorch tensors
    train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    test_data = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

    # DataLoader for batching
    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

    # Model initialization
    model = NeuralNetwork(input_size=input_size, hidden_size=64, output_size=output_size)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()  # For multi-class classification
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    for epoch in range(20):  # You can increase the number of epochs for better results
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Print the training loss for every epoch
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

    # Evaluation
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.numpy())
            y_true.extend(labels.numpy())

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")

# Main function
def main(csv_file):
    # Load and preprocess the data
    features_scaled, target = load_and_preprocess_data(csv_file)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

    # Define input and output sizes
    input_size = X_train.shape[1]  # Number of features
    output_size = len(set(y_train))  # Number of unique labels

    # Train and evaluate the neural network
    train_and_evaluate_neural_network(X_train, X_test, y_train, y_test, input_size, output_size)

if __name__ == "__main__":
    # Take the CSV filename as input
    csv_file = "d2_model_dataset.csv"  # Change to your file path
    main(csv_file)


RuntimeError: CUDA error: named symbol not found
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [88]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available


True
